# Topic Modeling Visualisierung der Amtspresse mit Python

Das folgende Notebook ist entstanden in Anlehnung an:

Blogpost:

https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0

Notebooks:

https://github.com/kapadias/mediumposts/blob/master/nlp/published_notebooks/Introduction%20to%20Topic%20Modeling.ipynb

https://github.com/bmabey/pyLDAvis/blob/master/notebooks/sklearn.ipynb


### Topic Modeling with LDA Implementation

1. Loading data
2. Data cleaning
3. Convert to Document Term Matrix
4. Train LDA Model
5. Create Visualization

## Loading data

In [1]:
# Importing modules
import pandas as pd
import re

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import warnings
warnings.simplefilter("ignore", DeprecationWarning)


In [3]:
# Read data 
path_to_csv_file = ### path to csv file
df = pd.read_csv(path_to_csv_file)
df.head()

,filename,full_date,year,year_month,article_count,article_counts_per_issue,article_length_chars,article_length_words,newspaper,headline,article_text
0,1863-07-01_01.txt,1863-07-01,1863,1863-07,1,9,7101,955,PC,Ist das Abgeordnetenhaus eine Obrigkeit?,Ist das Abgeordnetenhaus eine Obrigkeit?\nEin ...
1,1863-07-01_02.txt,1863-07-01,1863,1863-07,2,9,1083,129,PC,Wochenschau.,Wochenschau.\nDie Nachrichten aus Karlsbad übe...
2,1863-07-01_03.txt,1863-07-01,1863,1863-07,3,9,546,66,PC,Wochenschau.,Ihre Majestät die Königin Augusta verweilt noc...
3,1863-07-01_04.txt,1863-07-01,1863,1863-07,4,9,2145,310,PC,Wochenschau.,Se. Königl. Hoheit der Kronprinz hat von Litth...
4,1863-07-01_05.txt,1863-07-01,1863,1863-07,5,9,1904,260,PC,Wochenschau.,Es muß der Regierung zu großer Befriedigung ge...


## Data Cleaning


In [4]:
# Remove the columns
columns_to_drop = ['filename', 'article_count', 'article_counts_per_issue', 'article_length_chars', 'article_length_words']
df = df.drop(columns=columns_to_drop, axis=1)

df.head()

,full_date,year,year_month,newspaper,headline,article_text
0,1863-07-01,1863,1863-07,PC,Ist das Abgeordnetenhaus eine Obrigkeit?,Ist das Abgeordnetenhaus eine Obrigkeit?\nEin ...
1,1863-07-01,1863,1863-07,PC,Wochenschau.,Wochenschau.\nDie Nachrichten aus Karlsbad übe...
2,1863-07-01,1863,1863-07,PC,Wochenschau.,Ihre Majestät die Königin Augusta verweilt noc...
3,1863-07-01,1863,1863-07,PC,Wochenschau.,Se. Königl. Hoheit der Kronprinz hat von Litth...
4,1863-07-01,1863,1863-07,PC,Wochenschau.,Es muß der Regierung zu großer Befriedigung ge...


## Remove punctuation/lower casing

In [5]:
with open('german_stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.read()


In [ ]:
nlp = spacy.load('de_core_news_sm')

def lemmatizer(text, nlp):
    '''Lemmatize words.
    Input: Text
    Return: lemmatized text'''
        
    doc = nlp(str(text))
    return ' '.join([token.lemma_ for token in doc])

In [ ]:
%%time
# Lemmatization
df.loc[:,'article_text_processed'] = df.loc[:,'article_text'].apply(lambda x: lemmatizer(x, nlp))

# Print out the first rows of papers
df.loc[:,'article_text_processed'].head(15)                                                                     

In [ ]:
df.to_pickle('article_text_lemmatized.p')

In [6]:
# Remove punctuation
df.loc[:,'article_text_processed'] = df.loc[:,'article_text'].map(lambda x: re.sub('[,\.!?:;-]', '', str(x)))

# Replace \n with empty space
df.loc[:,'article_text_processed'] = df.loc[:,'article_text_processed'].map(lambda x: x.replace('\n', ' '))

# Convert the titles to lowercase
df.loc[:,'article_text_processed'] = df.loc[:,'article_text_processed'].map(lambda x: x.lower())

# Remove Stopwords
df.loc[:,'article_text_processed'] = df.loc[:,'article_text_processed'].apply(lambda x: ' '.join([item for item in x.split() if item not in stopwords]))

# w for w in word_tokens if not w in stop_words
# apply(lambda x: [item for item in x if item not in stopwords])

# Print out the first rows of papers
df.loc[:,'article_text_processed'].head()

<>:2: DeprecationWarning: invalid escape sequence \.
<>:2: DeprecationWarning: invalid escape sequence \.
<>:2: DeprecationWarning: invalid escape sequence \.
<ipython-input-6-be64af15204e>:2: DeprecationWarning: invalid escape sequence \.
  df.loc[:,'article_text_processed'] = df.loc[:,'article_text'].map(lambda x: re.sub('[,\.!?:;-]', '', str(x)))


0    abgeordnetenhaus obrigkeit weithin bekannter g...
1    wochenschau nachrichten karlsbad befinden unse...
2    majestät königin augusta verweilt england besu...
3    königl hoheit kronprinz litthauen reise fortge...
4    regierung befriedigung gereichen preßverordnun...
Name: article_text_processed, dtype: object

## Convert to Document Term Matrix

In [7]:
docs_raw = df.loc[:,'article_text_processed']

print(len(docs_raw))
print(type(docs_raw))

29142
<class 'pandas.core.series.Series'>


In [8]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(docs_raw)
print(dtm_tf.shape)

(29142, 34762)


## Train LDA-Model

In [9]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tf.fit(dtm_tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=20, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [10]:
# Help function for printing Topics
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

print("Topics found via LDA:")
print_topics(lda_tf, tf_vectorizer, 10)

Topics found via LDA:

Topic #0:
industrie lage mark wirthschaftlichen ausfuhr jahres preise handel einfuhr letzten

Topic #1:
herr freisinnigen herren partei richter sagen herrn lassen politik weise

Topic #2:
deutschen deutschland deutsche staaten landwirthschaft frankreich zoll waaren getreide england

Topic #3:
kaiser majestat berlin bismarck kaisers konig deutschen kronprinzen schreiben gesellschaft

Topic #4:
arbeiter zahl personen kinder fallen namentlich einzelnen bestimmungen kreise arbeiten

Topic #5:
arbeiter socialdemokratie gesellschaft arbeit sozialdemokratie arbeitern socialdemokratischen socialdemokraten partei mark

Topic #6:
mark zahl landwirthschaft landwirthschaftlichen bevolkerung personen betrug theil landlichen namentlich

Topic #7:
kaiser majestat berlin konig prinzen kaiserin wilhelm prinz kaisers koniglichen

Topic #8:
regierung kirche partei katholischen parteien nationalliberalen wahlen liberalen staat fortschrittspartei

Topic #9:
berlin hoheren personen ve

## Create Visualiztion with pyLDAvis

In [12]:
vis = pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)
vis

C:\Users\Martin\Anaconda3\envs\topic_modeling\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.106896 -0.097653       1        1  11.232305
19     0.009185 -0.160137       2        1   9.362039
14    -0.150421 -0.033555       3        1   7.669057
13    -0.058562 -0.149755       4        1   7.468936
18     0.014838 -0.129677       5        1   6.752005
7      0.302567 -0.056063       6        1   6.458387
15    -0.135398  0.088951       7        1   6.227085
8     -0.094659 -0.121568       8        1   5.481792
16    -0.001318 -0.121916       9        1   4.785969
4     -0.065349  0.183254      10        1   4.624819
6     -0.100090  0.190587      11        1   4.492791
11     0.107847 -0.022790      12        1   4.021082
12    -0.052454  0.015741      13        1   3.615326
2     -0.127714  0.040173      14        1   3.502109
5     -0.054281  0.040623      15        1   2.981043
0     -0.114349  0.117532      16        1   2.732762
10     0.123245 -0.019488      17        1   2.553302
17     0.273250  0.219123      18        1   2.114420
3      0.176325 -0.055589      19        1   2.016655
9      0.054235  0.072205      20        1   1.908117, topic_info=      Category          Freq          Term         Total  loglift  logprob
16613  Default  17077.000000        kaiser  17077.000000  30.0000  30.0000
23287  Default  24127.000000     regierung  24127.000000  29.0000  29.0000
19405  Default  11267.000000          mark  11267.000000  28.0000  28.0000
1766   Default   7269.000000      arbeiter   7269.000000  27.0000  27.0000
19273  Default   7503.000000      majestat   7503.000000  26.0000  26.0000
...        ...           ...           ...           ...      ...      ...
6992   Topic20    273.707968     deutschen  18151.339564  -0.2354  -5.7905
4873   Topic20    214.194183  betreffenden   1834.123124   1.8116  -6.0357
31245  Topic20    231.407383    verwaltung   5902.353286   0.7201  -5.9584
10351  Topic20    216.114719         falle   2810.541413   1.3937  -6.0268
16534  Topic20    214.227867          juli   2651.616347   1.4432  -6.0355

[1736 rows x 6 columns], token_table=       Topic      Freq                   Term
term                                         
7          4  0.951596      abanderungsantrag
26         2  0.012900                  abend
26         4  0.000679                  abend
26         5  0.014937                  abend
26         6  0.761124                  abend
...      ...       ...                    ...
34590     17  0.000823             zuversicht
34590     19  0.003294             zuversicht
34648     10  0.993773        zwangserziehung
34658      6  0.010083  zwangsversteigerungen
34658     11  0.978045  zwangsversteigerungen

[8612 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 20, 15, 14, 19, 8, 16, 9, 17, 5, 7, 12, 13, 3, 6, 1, 11, 18, 4, 10])

In [13]:
pyLDAvis.save_html(vis, 'pylda_amtspresse.html')

## Convert to Document Term Matrix with Tf-Idf

In [ ]:
df = pd.read_pickle('article_text_processed.p')

In [ ]:
docs_raw = df.loc[:,'article_text_processed']

print(len(docs_raw))
print(type(docs_raw))

In [ ]:
tfidf_vectorizer = TfidfVectorizer(strip_accents = 'unicode',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tfidf = tfidf_vectorizer.fit_transform(docs_raw)
print(dtm_tfidf.shape)

In [ ]:
with open('dtm_tfidf_vectorizer.p', 'wb') as f:
    pickle.dump(dtm_tfidf, f)

In [ ]:
%%time
lda_tfidf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tfidf.fit(dtm_tfidf)

In [ ]:
with open('lda_tfidf_model.p', 'wb') as f:
    pickle.dump(lda_tfidf, f)

In [ ]:
print("Topics found via LDA:")
print_topics(lda_tfidf, tfidf_vectorizer, 10)